<a href="https://colab.research.google.com/github/mehrn79/npl_wrod2wec/blob/main/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **word2vec2 implematation**

## **preprocessing**

### **libraries for preprocessing**

In [ ]:
pip install hazm

     |████████████████████████████████| 316 kB 5.0 MB/s 
     |████████████████████████████████| 1.4 MB 56.9 MB/s 
     |████████████████████████████████| 233 kB 45.1 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394487 sha256=7a67e3b7cbbb6e79ec9ad4c4208091564dcbe52886a9ad80163b9fab7e8d9bb6
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154066 sha256=4f4642ecdb86df45acc68c338b9cddfba9f80cd643fc3495cb6fe34e77be7092
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [ ]:
from __future__ import unicode_literals
from hazm import *
from keras.preprocessing import text
from collections import Counter

### **getting the corpes & stop words**

In [ ]:
with open('/content/shams.txt') as f:
  lines = f.readlines()

with open('/content/stopwords.txt') as file:
  stopLines = file.readlines()
stopWord = [item.replace('\n',"") for item in stopLines]
  

### **cleaning the corpes**

In [ ]:
words= []
stemmer = Stemmer()
for sent in lines :
  words.append(word_tokenize(sent))

text_word_list=[]
corpes=[]
new_words=[]

for wordSent in words :
  cleanSent=[]
  for word in wordSent :
    if word not in stopWord :
      cleanSent.append(stemmer.stem(word))
  if(len(cleanSent)>1) :
    text_word_list+=cleanSent
    new_words.append(cleanSent)

value_of_repetition = dict(Counter(text_word_list))
repetition=[]
for word in value_of_repetition :
  if value_of_repetition.get(word)==1:
    repetition.append(word)
clear_word_list=[x for x in text_word_list if x not in repetition]

text_word_list=[]

for wordSent in new_words :
  cleanSent=[]
  for word in wordSent :
    if word not in repetition :
      cleanSent.append(word)
  if(len(cleanSent)>1) :
    text_word_list+=cleanSent
    corpes.append(' '.join(cleanSent))

### **tokenizing the corpes**

In [ ]:
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(corpes)

word2id = tokenizer.word_index
id2word = {v:k for k, v in word2id.items()}


### **creating list of target words and context words**

In [ ]:
window = 2
word_lists = []

for text in corpes:
    text=word_tokenize(text)

    for i, word in enumerate(text):
        for w in range(window):
            if i + 1 + w < len(text): 
                word_lists.append([word] + [text[(i + 1 + w)]])   
            if i - w - 1 >= 0:
                word_lists.append([word] + [text[(i - w - 1)]])

### **finding unique words**

In [ ]:
words_set = list(set(text_word_list))
words_set.sort()

unique_word_dict = {}
for i, word in enumerate(words_set):
  unique_word_dict.update({
    word: i
  })

## **make X & Y of train**

In [ ]:
from scipy import sparse
import numpy as np
from tqdm import tqdm

n_words = len(unique_word_dict)
 
words = list(unique_word_dict.keys())

X = []
Y = []

for i, word_list in tqdm(enumerate(word_lists)):
    main_word_index = unique_word_dict.get(word_list[0])
    context_word_index = unique_word_dict.get(word_list[1])
 
    X_row = np.zeros(n_words)
    Y_row = np.zeros(n_words)

    X_row[main_word_index] = 1
    Y_row[context_word_index] = 1

    X.append(X_row)
    Y.append(Y_row)

X = np.asarray(X)
Y = np.asarray(Y)


110306it [00:04, 22552.61it/s]


## **creating training model**

In [ ]:
from keras.models import Input, Model
from keras.layers import Dense

embed_size = 15

inp = Input(shape=(X.shape[1],))
x = Dense(units=embed_size, activation='linear')(inp)
x = Dense(units=Y.shape[1], activation='softmax')(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss = 'mean_squared_error', optimizer = 'Adam', metrics=['categorical_accuracy'])
model.summary()
model.fit(
    x=X, 
    y=Y, 
    epochs=20,
    verbose=1
    )



Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3447)]            0         
                                                                 
 dense (Dense)               (None, 15)                51720     
                                                                 
 dense_1 (Dense)             (None, 3447)              55152     
                                                                 
Total params: 106,872
Trainable params: 106,872
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
3448/3448 [==============================] - 19s 5ms/step - loss: 2.9003e-04 - categorical_accuracy: 0.0221
Epoch 2/20
3448/3448 [==============================] - 19s 6ms/step - loss: 2.9003e-04 - categorical_accuracy: 0.0249
Epoch 3/20
3448/3448 [==============================] - 19s 6ms/step - loss: 2.90

## **Creating a dictionary to store the embeddings**

In [ ]:
weights = model.get_weights()[0]

embedding_dict = {}
for word in words: 
    embedding_dict.update({
        word: weights[unique_word_dict.get(word)]
        })

[-0.09252767  0.06050421  0.00355326  0.13323204  0.0607237   0.02255718
  0.09553705  0.03446832 -0.00841697  0.06273373 -0.07020693  0.0751634
  0.0452446  -0.08023857  0.1716735 ]


## **return similar words**

In [66]:
from sklearn.metrics.pairwise import euclidean_distances

def the_most_similar_words(word) :
  distance_matrix = euclidean_distances(weights)
  similar_words = {search_term: [list(unique_word_dict.keys())[idx] for idx in distance_matrix[unique_word_dict[search_term]-1].argsort()[1:15]+1] 
                   for search_term in [word]}

  return similar_words

In [69]:
the_most_similar_words('شمس')

{'شمس': ['نیاف',
  'قمارخانه',
  'همتا',
  'زشت',
  'گریس',
  'پرگهر',
  'خریدار',
  'رمد',
  'جرم',
  'اشتاب',
  'خموش',
  'مسل',
  'همنفس',
  'رطل']}